In [30]:
import pymysql
from bs4 import BeautifulSoup as bs
from selenium import webdriver
from time import sleep

In [31]:
conn = pymysql.connect(host='localhost', user='root', password='1234', db='java02_kdd')
cursors = conn.cursor()

In [33]:
sql = 'select idx, lyrics from song where album = %s and idx > 528'
cursors.execute(sql, '')
result = cursors.fetchall()

In [34]:
idx_list = []

for item in result:
    idx_list.append(item[0])
    
len(idx_list)

7

In [35]:
url = 'https://search.naver.com/search.naver?where=nexearch&sm=top_hty&fbm=1&ie=utf8&query='
driver = webdriver.Chrome('./95/chromedriver.exe')
driver.implicitly_wait(10)
cnt = 0
    
for idx in idx_list:
    sql = 'select idx, title, artist from song where idx = %s'
    cursors.execute(sql, idx)
    result = cursors.fetchall()
    query = result[0][1] + ' - ' + result[0][2]
    driver.get(url + query)
    soup = bs(driver.page_source, 'html.parser')
    
    if soup.select('ul.tab_list li.tab a') == []:
        driver.get(url + result[0][1])
        soup = bs(driver.page_source, 'html.parser')
    
        if soup.select('ul.tab_list li.tab a') == []:
            print(result[0][1], '검색 실패')
            continue
        
    html = soup.select('ul.tab_list a')[1].get('href')
    key = 'https://search.naver.com/search.naver' + html
    
    driver.get(key)
    soup = bs(driver.page_source, 'html.parser')

    if soup.select('div.info_group dd') !=[]:
        data = soup.select('div.info_group dd')
        cnt = len(data)

        album = ''
        releaseDate = ''
        genre = ''
        music = ''
        words = ''
        arranger = ''

        if cnt == 7:
            artist = data[0].text.strip()
            album = data[1].text.strip()
            releaseDate = data[2].text.strip()
            genre = data[3].text.strip()
            music = data[4].text.strip()
            words = data[5].text.strip()
            arranger = data[6].text.strip()
        elif cnt == 6:
            artist = data[0].text.strip()
            album = data[1].text.strip()
            releaseDate = data[2].text.strip()
            genre = data[3].text.strip()
            music = data[4].text.strip()
            words = data[5].text.strip()
        elif cnt == 5:
            artist = data[0].text.strip()
            album = data[1].text.strip()
            releaseDate = data[2].text.strip()
            genre = data[3].text.strip()
            music = data[4].text.strip()
        elif cnt == 4:
            artist = data[0].text.strip()
            album = data[1].text.strip()
            releaseDate = data[2].text.strip()
            genre = data[3].text.strip()

        lyrics_data = soup.select_one('p.no_ellipsis')
        lyrics = ""
        no = 0
        for item in lyrics_data:
            if no % 2 == 0:
                lyrics += item.strip()
                no += 1
            else:
                lyrics += '\n'
                no += 1
        
        try:
            sql = 'update song set album = %s, releaseDate = %s, genre = %s, music = %s, words = %s, arranger = %s, lyrics = %s where idx = %s'
            val = (album, releaseDate, genre, music, words, arranger, lyrics, idx)
            cursors.execute(sql, val)

            conn.commit()
            cnt += 1
        except:
            print('db오류')
        
print('insert row : ', cnt)
driver.close()
cursors.close()
conn.close()
        

<ipython-input-35-7501bf8fb188>:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('./95/chromedriver.exe')


PYRAMID (feat.릴보이(lIlBOI), 개코) 검색 실패
Sunshine 검색 실패
insert row :  8


In [8]:
cursors.close()
conn.close()